<a href="https://colab.research.google.com/github/andrewjason21/repo/blob/main/Copy_of_Deep_Q_Learning_for_Lunar_Landing_Partial_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Q-Learning for Lunar Landing

## Part 0 - Installing the required packages and importing the libraries

### Installing Gymnasium

In [1]:
!pip install gymnasium
!pip install "gymnasium[atari, accept-rom-license]"
!apt-get install -y swig
!pip install gymnasium[box2d]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 9.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 54.6 MB/s eta 0:00:00
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.6.1-py3-none-any.whl size=446661 sha256=78e795583209626f94e72f7e6475cfac6239bc3d2b607835001d08503f3aa789
  Stored in directory: /root/.cache/pip/wheels/6b/1b/ef/a43ff1a2f1736d5711faa1ba4c1f61be1131b8899e6a057811
Successfully built AutoROM.accept-rom-license
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  swig4.0
Suggested packages:
  swig-doc swig-examples swig4.0-examples swig4.0-doc
The following NEW packages will be installed:
  

### Importing the libraries

In [ ]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.autograd as autograd
from torch.autograd import Variable
from collections import deque, namedtuple

## Part 1 - Building the AI

### Creating the architecture of the Neural Network

In [ ]:
class Network(nn.Module):

  def __init__(self, state_size,action_size,seed=42):
      super(Network,self).__init__()
      self.seed=torch.manual_seed(seed)
      self.fc1=nn.linear(state_size,64)
      self.fc2=nn.linear(64,64)
      self.fc3=nn.linear(64,action_size)

      def forward (self,state):
        x=self.fc1(state)
        x=F.relu(x)
        x=self.fc2(x)
        x=F.relu(x)
        return self.fc3(x)

## Part 2 - Training the AI

### Setting up the environment

> Indented block



In [ ]:
import gymnasium as gym
env=gym.make('LunarLander-v2')
state_shape=env.observation_space.shape
state_size=env.observation_space.shape[0]
number_actions=env.action_space.n
print('state_shape: ',state_shape)
print('state_size: ',state_size)
print('number_actions: ',number_actions)


state_shape:  (8,)
state_size:  8
number_actions:  4


### Initializing the hyperparameters

In [ ]:
learning_rate=5e-4
minibatch_size=100
discount_factor=0.99
replay_buffer_size=int(1e5)
interpolation_parameter=1e-3

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Implementing Experience Replay

In [ ]:
class ReplayMemory(object):

 def __init__(self,capacity):
     self.device=torch.device("cuda:0"if torch.cuda.is_avalaible() else "cpu")
     self.capacity=capacity
     self.memory=[]

def push(self,event):
  self.memory.append(event)
  if len(self.memory)>self.capacity:
    del self.memory[0]

def sample(self,batch_size):
 experiences=random.sample(self.memory,k=batch_size)
 states=torch.from_numpy(np.vstack([e[0]for e in experiences if e is not None])).float().to(self.device)
 actions=torch.from_numpy(np.vstack([e[1]for e in experiences if e is not None])).long().to(self.device)
 rewards=torch.from_numpy(np.vstack([e[2]for e in experiences if e is not None])).float().to(self.device)
 next_states=torch.from_numpy(np.vstack([e[3]for e in experiences if e is not None])).float().to(self.device)
 dones=torch.from_numpy(np.vstack([e[4]for e in experiences if e is not None])).astype(np.uint8).float().to(self.device)
 return states,next-states,actions,rewards,dones

### Implementing the DQN class

In [ ]:
class Agent():
 def __init__(self,state_size,action_size):
  self.device=torch.device("cuda:0"if torch.cuda.is_avalaible() else "cpu")
  self.state_size=state_size
  self.action_size=action_size
  self.local_qnetwork=Newtork(state_size,action_size),to(self.device)
  self.target_qnetwork=Newtork(state_size,action_size),to(self.device)
  self.optimizer=optime.Adam(self.local_qnetwork.parameters(),lr=learning_rate)
  self.memory=ReplayMemory(replay_buffer_size)
  self.step=0

### Initializing the DQN agent

### Training the DQN agent

## Part 3 - Visualizing the results

In [2]:
import glob
import io
import base64
import imageio
from IPython.display import HTML, display
from gym.wrappers.monitoring.video_recorder import VideoRecorder

def show_video_of_model(agent, env_name):
    env = gym.make(env_name, render_mode='rgb_array')
    state, _ = env.reset()
    done = False
    frames = []
    while not done:
        frame = env.render()
        frames.append(frame)
        action = agent.act(state)
        state, reward, done, _, _ = env.step(action.item())
    env.close()
    imageio.mimsave('video.mp4', frames, fps=30)

show_video_of_model(agent, 'LunarLander-v2')

def show_video():
    mp4list = glob.glob('*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")

show_video()

NameError: name 'agent' is not defined